In [6]:
import plotly.graph_objects as go
from IPython.display import display

def init_real_time_plot():
    # global fig_x, fig_y, fig_z
    global fig_y, fig_z

    # Initialize figures with two traces each: actual values and moving averages
    # fig_x = go.FigureWidget(data=[
    #     go.Scatter(x=[], y=[], mode='lines+markers', name='X Values'),
    #     go.Scatter(x=[], y=[], mode='lines', name='X Moving Avg', line=dict(color='red'))])
    fig_y = go.FigureWidget(data=[
        go.Scatter(x=[], y=[], mode='lines+markers', name='Y Values'),
        go.Scatter(x=[], y=[], mode='lines', name='Y Moving Avg', line=dict(color='red'))])
    fig_z = go.FigureWidget(data=[
        go.Scatter(x=[], y=[], mode='lines+markers', name='Z Values'),
        go.Scatter(x=[], y=[], mode='lines', name='Z Moving Avg', line=dict(color='red'))])

    # Set titles and display the figures
    # fig_x.update_layout(title_text='Real-Time X Values')
    fig_y.update_layout(title_text='Real-Time Y1 Values')
    fig_z.update_layout(title_text='Real-Time Y3 Values')
    # display(fig_x, fig_y, fig_z)
    display(fig_y, fig_z)




In [9]:
def send_alert_to_microbit(serial_connection, message):
    """
    Sends an alert message to the microbit via the serial connection.

    Args:
    - serial_connection: The open serial connection to the microbit.
    - message: The message to send as a string.
    """
    serial_connection.write(message.encode())
    # serial_connection.close()

In [4]:
import serial
import time
import plotly.graph_objects as go
from plotly.subplots import make_subplots

def read_serial_data(port, baud_rate=115200, timeout=1, duration=10):
    
    global ser
    init_real_time_plot()  # Ensure you have initialized a Plotly FigureWidget globally
    ser = serial.Serial(port, baud_rate, timeout=timeout)
    end_time = time.time() + duration
    value_index = 0  # Cycle through 0 (x), 1 (y), 2 (z)
    sample_index = 0  # Increment for each new set of x, y, z values

    lst_y1 = []
    lst_y3 = []
    try:
        while True: # if want to read data continuously
        # while time.time() < end_time:  # Continuously read and plot data
            if ser.in_waiting > 0:
                data = ser.readline().decode('utf-8').strip()
                if data:
                    try:
                        value = int(data)
                        print(data,value)
                        # Determine which plot to update based on value_index
                        # if value_index == 0:  # X value
                        #     add_data_to_plot(ser, value, None, None, sample_index)
                        # elif value_index == 1:  # Y value
                        #     add_data_to_plot(ser, None, value, None, sample_index)
                        
                        if value_index == 0:  # X value                                
                            # add_data_to_plot(ser, None, value, None, sample_index)
                            lst_y1.append(value)
                        elif value_index == 1:  # Y value
                            # add_data_to_plot(ser, None, None, value, sample_index)
                        # elif value_index == 2:  # Z value
                        #     add_data_to_plot(ser, None, None, value, sample_index)
                            lst_y3.append(value)
                            # sample_index += 1  # Increment sample_index after a full cycle
                        
                        value_index = (value_index + 1) % 2  # Cycle through 0, 1, 2
                    except ValueError:
                        pass  # Ignore non-integer values
                time.sleep(0.05)  # Adjust based on your data rate
                if len(lst_y1)>0 and len(lst_y3)>0:
                    if lst_y1[-1] <-900 and lst_y3[-1]<-960:
                        send_alert_to_microbit(ser, "Alertttt")
                    
    finally:
        ser.close()


In [56]:
import serial
import time
import plotly.graph_objects as go
from plotly.subplots import make_subplots

def read_serial_data(port, baud_rate=115200, timeout=1, duration=10):
    
    global ser
    init_real_time_plot()  # Ensure you have initialized a Plotly FigureWidget globally
    ser = serial.Serial(port, baud_rate, timeout=timeout)
    end_time = time.time() + duration
    value_index = 0  # Cycle through 0 (x), 1 (y), 2 (z)
    sample_index = 0  # Increment for each new set of x, y, z values

    lst_y1 = []
    lst_y3 = []
    try:
        while True: # if want to read data continuously
        # while time.time() < end_time:  # Continuously read and plot data
            if ser.in_waiting > 0:
                data = ser.readline().decode('utf-8').strip()
                if data:
                    try:
                        value = int(data)
                        # print(data,value)
                        # Determine which plot to update based on value_index
                        # if value_index == 0:  # X value
                        #     add_data_to_plot(ser, value, None, None, sample_index)
                        # elif value_index == 1:  # Y value
                        #     add_data_to_plot(ser, None, value, None, sample_index)
                        
                        if value_index == 0:  # X value                                
                            add_data_to_plot(ser, None, value, None, sample_index)
                            lst_y1.append(value)
                        elif value_index == 1:  # Y value
                            add_data_to_plot(ser, None, None, value, sample_index)
                            lst_y3.append(value)
                        # elif value_index == 2:  # Z value
                        #     add_data_to_plot(ser, None, None, value, sample_index)
                            sample_index += 1  # Increment sample_index after a full cycle
                        
                        value_index = (value_index + 1) % 2  # Cycle through 0, 1, 2
                    except ValueError:
                        pass  # Ignore non-integer values
                # time.sleep(0.01)  # Adjust based on your data rate
                if len(lst_y1)>1 and len(lst_y3)>1:
                    if lst_y1[-1] <-900 and lst_y1[-2]>-900 and any(value < -900 for value in lst_y3[-50:]):
                        print('alerted', lst_y1[-10:], lst_y3[-50:])
                        send_alert_to_microbit(ser, "Alertttt")
                if len(lst_y1) > 100: 
                    del lst_y1[:50]
                if len(lst_y3) > 100:
                    del lst_y3[:50]
                
                if len(lst_y1) >= 10:
                    dynamic_list = lst_y1[-10:]
                    gradient = (dynamic_list[-1] - dynamic_list[0])/10
                    # print('dynamic list', dynamic_list, gradient)
                    if gradient < -95:
                        print('NO')
                        send_alert_to_microbit(ser, "Too fast!")
                    if gradient < -20 and gradient >= -95:
                        print('yes')
                        send_alert_to_microbit(ser, "The speed is good")
                
    finally:
        ser.close()


In [57]:
# Define window size for the moving average
N = 20 # Example window size
# Initialize rolling windows for x, y, z values
rolling_window_x, rolling_window_y, rolling_window_z = [], [], []

THRESHOLD_X = -150
THRESHOLD_Y = -960
THRESHOLD_Z = -900
data = {
    'x': [],
    'y': [],
    'z': [],
    'x_moving_avg': [],
    'y_moving_avg': [],
    'z_moving_avg': []
}

def add_data_to_plot(serial_connection, x_new, y_new, z_new, sample_index):
    resp = ''
    # global rolling_window_x, rolling_window_y, rolling_window_z
    global rolling_window_y, rolling_window_z

    
    # Update and calculate moving average for X values, check threshold
    # if x_new is not None:
    #     rolling_window_x.append(x_new)
    #     rolling_window_x = rolling_window_x[-N:]
    #     x_moving_avg = sum(rolling_window_x) / len(rolling_window_x)
        
        
        
    #     with fig_x.batch_update():
    #         fig_x.data[0].x += (sample_index,)
    #         fig_x.data[0].y += (x_new,)
    #         fig_x.data[1].x += (sample_index,)
    #         fig_x.data[1].y += (x_moving_avg,)
        
    #     if x_moving_avg > THRESHOLD_X:
    #         # print('alertx')
    #         send_alert_to_microbit(serial_connection, "Alert X")

    # Repeat the process for Y and Z values
    if y_new is not None:
        # rolling_window_y.append(y_new)
        # rolling_window_y = rolling_window_y[-N:]
        # y_moving_avg = sum(rolling_window_y) / len(rolling_window_y)
        
        with fig_y.batch_update():
            fig_y.data[0].x += (sample_index,)
            fig_y.data[0].y += (y_new,)
            # fig_y.data[1].x += (sample_index,)
            # fig_y.data[1].y += (y_moving_avg,)
            # Ensure only the last 100 data points are retained
            fig_y.data[0].x = fig_y.data[0].x[-100:]  # Keep last 100 x values
            fig_y.data[0].y = fig_y.data[0].y[-100:]  # Keep last 100 y values
            
        # if rolling_window_y[-1] < THRESHOLD_Y:
            
        # if y_new < THRESHOLD_Y:
        #     # print('alerty')
        #     send_alert_to_microbit(serial_connection, "Alert Y")
            
    if z_new is not None:
        # rolling_window_z.append(z_new)
        # rolling_window_z = rolling_window_z[-N:]
        # z_moving_avg = sum(rolling_window_z) / len(rolling_window_z)
        # z_moving_avg = sum(rolling_window_z) / len(rolling_window_z)
        
        with fig_z.batch_update():
            fig_z.data[0].x += (sample_index,)
            fig_z.data[0].y += (z_new,)
            # fig_z.data[1].x += (sample_index,)
            # fig_z.data[1].y += (z_moving_avg,)
            # Ensure only the last 100 data points are retained
            fig_z.data[0].x = fig_z.data[0].x[-100:]  # Keep last 100 x values
            fig_z.data[0].y = fig_z.data[0].y[-100:]  # Keep last 100 y values
        
        
        # if z_new < THRESHOLD_Z:
        #     # print('alertz')
        #     send_alert_to_microbit(serial_connection, "Alert Z")

    #if rolling_window_y[-1] < THRESHOLD_Y and sum(rolling_window_y[-10:])/10 > -800 and sum(rolling_window_z[-10:])/10 < THRESHOLD_Z:
    #    print('test')
    #    send_alert_to_microbit(ser, "Alertttt")
    # print(rolling_window_y)
    # if rolling_window_y[-1] < THRESHOLD_Y:
    # # if rolling_window_y[-1] < THRESHOLD_Y and sum(rolling_window_y[-10:])/10 > -300:
    #     print('last 10', rolling_window_y[-10:])
    #     send_alert_to_microbit(serial_connection, "Alertttt")

In [58]:
port = '/dev/tty.usbmodem21102'  # Example for Linux/Mac. Use 'COM#' for Windows.
read_serial_data(port, duration=15)  # Read data for 10 seconds


FigureWidget({
    'data': [{'mode': 'lines+markers',
              'name': 'Y Values',
              'type': 'scatter',
              'uid': '30126a74-c5aa-45c6-aea8-7654f4e4828e',
              'x': [],
              'y': []},
             {'line': {'color': 'red'},
              'mode': 'lines',
              'name': 'Y Moving Avg',
              'type': 'scatter',
              'uid': 'af9ceaed-adb1-48b0-8ed4-a8b4a5af142d',
              'x': [],
              'y': []}],
    'layout': {'template': '...', 'title': {'text': 'Real-Time Y1 Values'}}
})

FigureWidget({
    'data': [{'mode': 'lines+markers',
              'name': 'Z Values',
              'type': 'scatter',
              'uid': '7d10838a-ee38-4bf7-85fe-4b576c7c2a7f',
              'x': [],
              'y': []},
             {'line': {'color': 'red'},
              'mode': 'lines',
              'name': 'Z Moving Avg',
              'type': 'scatter',
              'uid': '5bc1cf78-efbe-4f3d-b57e-4b0c6f74a772',
              'x': [],
              'y': []}],
    'layout': {'template': '...', 'title': {'text': 'Real-Time Y3 Values'}}
})

yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
yes
yes
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
alerted [-516, -516, -516, -516, -516, -516, -516, -516, -516, -908] [-680, -680, -680, -680, -680, -680, -680, -680, -680, -680, 680, 680, -680, 680, 680, -1372, -1372, -1372, -1372, -1372, 1372, 1372, -1372, -1372, -1372, -1372, -1372, -1372, -172, -1372, -1372, -1372, 1372, -1372, -1372, -760, -760, -760, -760, -760, -760, -760, -760, -760, -760, -760, -760, -760, -760, -760]
yes
alerted [-516, -516, -516, -516, -516, -516, -516, -516, -516, -908] [-680, -680, -680, -680, -680, -680, -680, -680, -680, 680, 680, -680, 680, 680, -1372, -1372, -1372, -1372, -1372, 1372, 1372, -1372, -1372, -1372, -1372, -1372, -1372, -172, -

KeyboardInterrupt: 